<a href="https://colab.research.google.com/github/Louiii/Markov-Decision-Processes/blob/master/Markov_Decision_Process_Operations_Research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Markov Decision Processes

# Policy Improvement Algorithm

These dynamic programming examples do not have a definite final stage from which we work back via backwards induction. These are problems with no natural final stage: infinite horizon problems. These problems are only manageable if we impose some additional structure on how the stages progress; some version of the Markov property is what we need here. Markov chains can be used to model the evolution of systems that change in some random fashion which may depend on their current state but not on their earlier history. These are Markov decision processes, a useful and important class of models which allow us to take actions which at some known cost modify the transition probabilities of a system. The question of interest is how to optimally choose which actions to take.

In [0]:
import numpy as np

class MDP:
    def __init__(self, markov_chains, rewards, actions, states):
        self.m = len(states)
        self.states = states
        self.actions = actions
        expected_reward = []
        for action in actions:
            e = []
            for i in range(self.m):
                e.append(sum( [rewards[action][i][j] * markov_chains[action][i][j] for j in range(self.m)]) )
            expected_reward.append(e)
        self.expected_reward = np.array(expected_reward)
        self.markov_chains = markov_chains
        self.iteration_count = 0
        


    def policy_iteration(self, A):
        self.iteration_count += 1
        r, V = [], []
        for state in states:# for each state i...
            r.append( -self.expected_reward[A[state]][state] ) # rewards[A[i]][i, A[i]] 
            V.append( [markov_chains[A[state]][state, j] for j in states[:-1]] )
        V = np.array(V)  
        r = np.array(r)    
        X = np.zeros((self.m, self.m))
        for i in range(self.m): 
            X[i, -1] = -1
            X[i,  i] = -1
            for j in states[:-1]:
                X[i, j] += V[i, j]
        V = np.linalg.solve(X, r)
        lambda_ = V[-1]
        T = []
        for state in states:
            t=[]
            for action in actions:
                VP = sum( [markov_chains[action][state, j] * V[j] for j in states[:-1]] )
                t.append( self.expected_reward[action][state] + VP ) # rewards[A[state]][state, action]
            T.append(t)
        improved_policy = []
        for i in states:
            improved_policy.append( T[i].index(max(T[i])) )
        return improved_policy, lambda_
    
    def iterate(self, A):
        A1, lmbd = self.policy_iteration(A)
        while (A != A1):
            print('\n\nNew policy: '+str(A1)+' (iteration '+str(self.iteration_count)+')\n\n')
            A = A1
            A1, lmbd = self.policy_iteration(A)
        return A, lmbd

# Op Research Q.83

83 (*). 
A tennis player, on serve, has two attempts at serving to win the point, and on each of the first and second serve can either serve hard or safe. A hard serve has probability 1/4 of going in, and if it does go in the player has probability 2/3 of winning the point. A safe serve has probability 4/5 of going in, and if it does go in the player has probability 1/2 of winning the point.
The server wants to optimize their service strategy over a long sequence of serves using the following simple reward system: the player gains 1 by winning the point and −1 by losing it (including by serving a double fault, i.e., if a second serve does not go in). If the outcome of the point is yet to be decided (e.g. if a first serve goes out) there is no immediate reward.
Formulate this problem as a Markov decision process with two states (first or second serve) and two actions (hard or safe). Write down the transition matrices associated with each of the two actions, and calculate the expected immediate rewards r(i,k) on performing action k in state i.
For the expected rewards, you should use the partition theorem over whether the serve is in or out; for example, you
should find that $r (0, 0) = \frac{1}{4}(\frac{1}{3}\cdot (-1) + \frac{2}{3}\cdot (+1)) + \frac{3}{4}\cdot 0 = \frac{1}{12}$
The aim is to maximize the long-run average reward per service.
Find the optimal strategy by exhaustion over the four possible policies.

In [12]:
actions = {0, 1}# hard, soft
states  = [0, 1,2]# 1st serve, 2nd serve, In
P1 = np.array([[0, 3/4, 1/4],
               [3/4, 0, 1/4],
               [1,0,0]])
P2 = np.array([[0, 1/5, 4/5],
               [1/5,0, 4/5],
               [1,0,0]])
R1 = np.array([[0, 0, 0],
               [-1, 0, 0],
               [1/3, 0, 0]])
R2 = np.array([[0, 0, 0],
               [-1, 0, 0],
               [0, 0, 0]])
rewards = { 0: R1, 1: R2}
markov_chains = { 0: P1, 1: P2}
initial_policy = [0, 0, 0]# In state 0, do action 0. In state 1, do action 0.  In state 2, do action 0. 

mdp_Q83 = MDP(markov_chains, rewards, actions, states)
final_policy_Q83, lmbd = mdp_Q83.iterate(initial_policy)
print(final_policy_Q83)
print("Long run expected reward: "+str(lmbd))



New policy: [1, 1, 0] (iteration 1)


[1, 1, 0]
Long run expected reward: 0.12962962962962962


# Operations Research Example 9.1

The manager of a market garden knows that output over the growing season is affected greatly by soil conditions in spring. The income of the business depends on output but also running costs e.g. it is possible at some cost to improve the soil condition by testing it and using fertilizers to restore missing nutrients or improving drainage etc. To reflect the fact that it takes quite a while for changes of state to occur the manager will estimate revenue for each pair (current state, next state). Let’s keep the problem simple: there are only three soil conditions good, fair and bad which we’ll denote 0,1, 2; the possible actions on the soil are to leave or improve it which we’ll label 0,1. The transition probabilities from season to season are



P(0) = [P(i, j;0)] = \begin{bmatrix}
    0.2 & 0.5 & 0.3 \\0 & 0.5 & 0.5 \\0 & 0 & 1 
  \end{bmatrix}
  
 P(1) = [P(i, j;1)] = \begin{bmatrix}
    0.3 & 0.6 & 0.1 \\0.1 & 0.6 & 0.3 \\0.05 & 0.4 & 0.55 
  \end{bmatrix}

The one-year revenues are

R(0) = [R(i, j;0)] = \begin{bmatrix}
    7 & 6 & 5 \\0 & 5 & 1 \\0 & 0 & -1 
  \end{bmatrix}
  
 R(1) = [R(i, j;1)] = \begin{bmatrix}
    6 & 5 & -1 \\7 & 4 & 0 \\6 & 3 & -2 
  \end{bmatrix}


The manager would like to know when it’s best to improve the soil condition.

In [13]:
actions = {0, 1}#label actions starting from 0
states  = [0, 1, 2]#label states starting from 0
P1 = np.array([[0.2, 0.5, 0.3],
               [  0, 0.5, 0.5],
               [  0,   0,   1]])
P2 = np.array([[0.3, 0.6, 0.1],
               [0.1, 0.6, 0.3],
               [0.05,0.4,0.55]])
R1 = np.array([[7, 6, 3],
               [0, 5, 1],
               [0, 0,-1]])
R2 = np.array([[6, 5,-1],
               [7, 4, 0],
               [6, 3,-2]])
rewards = { 0: R1, 1: R2}
markov_chains = { 0: P1, 1: P2 }
initial_policy = [0, 0, 1]# In state 1, do action 1. In state 2, do action 1. 


mdp_Ex9_1 = MDP(markov_chains, rewards, actions, states)
final_policy_Ex9_1, lmbd = mdp_Ex9_1.iterate(initial_policy)
print( final_policy_Ex9_1 )
print("Long run expected reward: "+str(lmbd))



New policy: [1, 1, 1] (iteration 1)


[1, 1, 1]
Long run expected reward: 2.255932203389831


# Question 85:


In the quiz show The Missing Link questions are of type 1, 2, 3 or 4. The value of a type i question is 100 for i = 1, 200 for i = 2, 500 for i = 3, 1000 for i = 4. Your probability of correctly answering a question of type i is 1−(0.1×i).
After a correctly answered type i question (i < 4), you can either bank the value of that question (in which case the next question is of type 1), or wait for the next question (in which case you bank nothing but the next question is of type i + 1). If you correctly answer a type 4 question you automatically bank 1000 and return to type 1. If you answer incorrectly (or pass) you bank nothing and return to type 1.
You wish to maximise the long-run amount banked per question asked.
(i) Formulate this problem as a Markov decision process. Hint: Use states {1, 2, 3, 4} for the type of question about
to be asked, and four non-zero rewards (where reward = money in the bank).
(ii) Solve the problem using the policy improvement algorithm starting with the policy of banking before a question of type 4.

In [14]:
actions = {0, 1}#answer, bank
states  = [0, 1, 2, 3]#each question currently on
P1 = np.array([[0.1, 0.9, 0, 0],
               [0.2, 0, 0.8, 0],
               [0.3, 0, 0, 0.7],
               [1, 0, 0, 0]])
P2 = np.array([[1, 0, 0, 0],
               [1, 0, 0, 0],
               [1, 0, 0, 0],
               [1, 0, 0, 0]])
R1 = np.array([[0, 0, 0, 0],
               [0, 0, 0, 0],
               [0, 0, 0, 0],
               [600, 0, 0, 0]])
R2 = np.array([[0, 0, 0, 0],
               [100, 0, 0, 0],
               [200, 0, 0, 0],
               [500, 0, 0, 0]])
rewards = { 0: R1, 1: R2}
markov_chains = { 0: P1, 1: P2 }
initial_policy = [0, 0, 0, 1]



mdp_q85 = MDP(markov_chains, rewards, actions, states)
final_policy_q85, lmbd = mdp_q85.iterate(initial_policy)
print( final_policy_q85 )
print("Long run expected reward: "+str(lmbd))



New policy: [0, 0, 0, 0] (iteration 1)


[0, 0, 0, 0]
Long run expected reward: 96.7989756722151
